#### SCCAF clustering for Cai 2020 + 2022 healthy PBMCs

**Objective**: Run SCCAF clustering in separate environment to solve dependency issues

- **Developed by**: Mairi McClean
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- v230316

In [ ]:
import matplotlib.pyplot as plt
from SCCAF import SCCAF_assessment, plot_roc
y_prob, y_pred, y_test, clf, cvsm, acc = SCCAF_assessment(adata.X, adata.obs['leiden'], n = 100)
plot_roc(y_prob, y_test, clf, cvsm = cvsm, acc = acc)
plt.show()
sc.pl.umap(adata, frameon = False, color = ['leiden', 'status', 'CD74'], size = 0.8, legend_fontsize = 5, legend_loc = 'on data')
sc.pl.umap(adata, frameon = False, color = ['leiden', 'status', 'tissue', 'ADH7', 'CDH1', 'CD74', 'CD3E', 'MUC20', 'DUSP4', 'FOXJ1', 'MUC1', 'FOXI1'], size = 1, legend_fontsize = 5)
### Export clustered object
adata
caiy_healthy
# Making a hybrid anndata object using sections from both original anndata object and the cai_tb_gex object
adata_export = anndata.AnnData(X = caiy_healthy.X, var = caiy_healthy.var, obs = adata.obs, uns = adata.uns, obsm = adata.obsm, layers = caiy_healthy.layers, obsp = adata.obsp)
adata_export
adata_export.write('/Volumes/Lacie/data_lake/Mairi_example/processed_files/scvi/CaiY_healthy_scRNA_PBMC_mm230315_scVI-clustered.raw.h5ad')
